# Libraries

In [3]:
import pandas as pd 
import json
import random 
import time
import hashlib


from kafka import KafkaProducer
from confluent_kafka import Producer
import socket
import json
import os

from auth_kaf import conf

# Purchase Simulation 

### Our Ecommerce have additionaly stores in Argentina and Colombia, lets add some data to enrich our dataset of purchases

In [ ]:
cities = ['Cordoba','CABA','Mendoza','Rosario','Bogotá','Medellín','Cali','Bucaramanga','Barranquilla']
payment_online = ['Credit_card','Debit_card', 'Mercado_pago', 'PSE']
payment_store = ['Cash','Credit_card','Debit_card', 'Mercado_pago','Nequi','Daviplata']
source = ['Facebook','Instagram','Organic','Twitter','Influencer_1','Influencer_2','Influencer_3','Influencer_4']
status_purchase = ['COMPLETED','FAILED_CHECKOUT','FAILED_API_RESPONSE','INSUFICCIENT_FUNDS','COMPLETED','COMPLETED','COMPLETED','COMPLETED','FAILED_API_RESPONSE','INSUFICCIENT_FUNDS','USER_ERROR','FRAUD','COMPLETED','COMPLETED','COMPLETED']

In [ ]:
cor_coords = [(-31.41251939443787, -64.2057391281784),(-31.419567363029973, -64.18756698913873),(-31.42772016730141, -64.21260398812967)]
caba_coords = [(-34.57184348827202, -58.39956474229912),(-34.57141944137543, -58.418790817515145),(-34.58555317238583, -58.380338667083116), (-34.60067360282511, -58.41673088088486)]
men_coords = [(-32.901049900889696, -68.79609480792477),(-32.87086429109348, -68.88154225297224),(-32.89106461079145, -68.83874808366518)]
ros_coords = [(-32.92414738125918, -60.65958736997858),(-32.90681139215926, -60.682868150503104),(-32.91624547502892, -60.77483786832939)]
bog_coords = [(4.651600960108946, -74.12628850377475),(4.661984763443271, -74.13466548843223),(4.647569693587232, -74.10186525959672)]
buc_coords = [(7.099917472863198, -73.10730617492302),(7.07243688331635, -73.10525936227518)]
cali_coords = [(3.4287786094866717, -76.53749228193497),(3.4148803158142966, -76.54041613631288),(3.4164091379805432, -76.54751692551635)]
bar_coords = [(11.014167139030025, -74.82747678131524),(11.004041676077495, -74.83545204769058),(10.990580146449894, -74.78876005521157)]
mede_coords = [(6.163315879042265, -75.6052691935286),(6.17784573272914, -75.59141059178306),(6.198053256721469, -75.5733524126965)]

In [ ]:
# Get random payment method

def get_pay_method(source,status_purchase,payment_online,payment_store):
    
    if source == 'Organic':
        
        payment = random.choice(payment_store)
        status = 'COMPLETED'
        order_type = 'STORE'
        
    elif source != 'Organic':
        
        payment = random.choice(payment_online)
        status = random.choice(status_purchase)
        order_type = 'ONLINE'
    
    return payment,status,order_type
        
# Get coords of the city

def get_coords(city):
     
    if city == 'Bogotá':
        coords = random.choice(bog_coords)
    elif city == 'Bucaramanga':
        coords = random.choice(buc_coords)
    elif city == 'Cali':
        coords = random.choice(cali_coords)
    elif city == 'Barranquilla':
        coords = random.choice(bar_coords)
    elif city == 'Medellín':
        coords = random.choice(mede_coords)
    elif city == 'Cordoba':
        coords = random.choice(cor_coords)
    elif city == 'CABA':
        coords = random.choice(caba_coords)
    elif city == 'Mendoza':
        coords = random.choice(men_coords)
    elif city == 'Rosario':
        coords = random.choice(ros_coords)
        
    return coords 

### Sample Dataset of products list

In [7]:
df = pd.read_csv('..//data/ecommerce_products.csv')
df

,PRODUCT_NAME,PRICING,COMISION,BRAND,CATEGORY
0,Mac_Book_4GB,1350000.0,0.20,Apple,LAPTOPS
1,Mac_Book_8GB,2550000.0,0.25,Apple,LAPTOPS
2,Mac_Book_16GB,3500000.0,0.30,Apple,LAPTOPS
3,Mac_Book_32GB,4800000.0,0.35,Apple,GAMING
4,Notebook_HP_4GB,1150000.0,0.15,HP,LAPTOPS
5,Notebook_HP_8GB,1850000.0,0.18,HP,LAPTOPS
6,Notebook_HP_16GB,3890000.0,0.20,HP,LAPTOPS
7,Notebook_HP_32GB,6990000.0,0.25,HP,GAMING
8,Notebook_Lenovo_4GB,1850000.0,0.20,Lenovo,LAPTOPS
9,Notebook_Lenovo_8GB,3199000.0,0.28,Lenovo,LAPTOPS


### To Do's

*  Deploy a Kafka Cluster on Confluent Cloud 
* Create a Kafka topic
* Test Python simulation Script

## Config Kafka Cluster

In [6]:
conf

{'bootstrap.servers': 'YOUR CONFLUENT CLOUD CLUSTER SERVER',
 'security.protocol': 'SASL_SSL',
 'sasl.mechanisms': 'PLAIN',
 'sasl.username': 'YOUR API KEY',
 'sasl.password': 'YOUR SECRET'}

In [6]:
producer = Producer(conf)

# Send Message To Kafka Topic

In [ ]:
delivered_records = 0
topic = 'YOUR KAFKA TOPIC'    

start_timestamp = pd.Timestamp.now() 
end_timestamp = start_timestamp + pd.Timedelta(minutes=60) # Simulation last one hour

x = 1

data_purchase = []

while start_timestamp <= end_timestamp:
    
    date = pd.to_datetime('today').strftime("%Y-%m-%d %H:%M:%S")
    
    product = df['PRODUCT_NAME'][random.randint(0,14)]
    pricing =  df[df['PRODUCT_NAME']==product]['PRICING'].values[0]
    comision =  df[df['PRODUCT_NAME']==product]['COMISION'].values[0]
    brand = df[df['PRODUCT_NAME']==product]['BRAND'].values[0]
    category = df[df['PRODUCT_NAME']==product]['CATEGORY'].values[0]
    source_temp = random.choice(source)
    pay = get_pay_method(source_temp,status_purchase,payment_online,payment_store)
    city = random.choice(cities)
    
    purchase = {'purchase_ID': str(hashlib.sha256(f"{x} {product} {pricing} {comision} {date} {source_temp} {pay[1]}".encode('utf-8')).hexdigest())[:10], 
            'Product_name' : product,
            'Pricing':str(pricing),
            'Comision':str(comision),
            'Revenue' : str(round(pricing*comision,2)),
            'Payment_Mehtod':pay[0],
            'Status' : pay[1],
            'Order_Type' : pay[2],
            'City':city,
            'Latitud' : str(get_coords(city)[0]) ,
            'Longitud' :  str(get_coords(city)[1]),
            'Source':source_temp,
            'Brand' : brand,
            'Category': category,
            'Created_at': date}
    
  

    record_key = "Purchase_simulator"
    record_value =  json.dumps(purchase).encode('utf-8')
    print("Producing record: {}\t{}".format(record_key, record_value))
    producer.produce(topic, key=record_key, value=record_value)
    producer.poll(0)
    delivered_records += 1
    x += 1
    
    time.sleep(random.choice([1,1.5]))

    
    

producer.flush()

print('\n')
print('\n')
print("{} messages were produced to topic {}!".format(delivered_records, topic))
print('\n')
print('\n')